# BAX pore assembly

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
plt.style.use("./scripts/output/notebooks/custom_style.mplstyle")
import xarray as xr

from scripts.grid_class import GridParms
from scripts.tree_class import Tree
from scripts.output.output_helper import *
from scripts.reference_solutions.ssa_helper import SSASol

%matplotlib inline

In [ ]:
slice_vec = np.array([12, 9, 2, 1, 0, 0, 0, 0, 0, 50, 0])

## Convention

### Tree tensor integrator

#### Partition 0
(0 1 2)(((3 4 6 7)(5 8))(9 10))

#### Partition 1
((0 1 2)(3 4 5))((6 7 9)(8 10))

In [ ]:
tree = readTree("output/bax_p1_r5-15-25_i_tau5e-2/output_t2900.nc")
P_slice_tensor0, P_sum_tensor0 = tree.calculateObservables(0, slice_vec)
P_slice_tensor1, P_sum_tensor1 = tree.calculateObservables(1, slice_vec)

#### Partition 2
((0 1)(2 3 4))((5 6 7 8)(9 10))

#### Two partitions
(0 1 2 3 4)(5 6 7 8 9 10)

In [ ]:
tree = readTree("output/bax_pfull_r5_i_tau5e-2/output_t2900.nc")
P_slice_tensor_full, P_sum_tensor_full = tree.calculateObservables(0, slice_vec)

### Matrix integrator
Two partitions, $r=5$, explicit Euler with variable time step size ($\min_i({{\tau_i}})=1.0$) and 100 substeps

In [ ]:
with xr.open_dataset("output/bax_matrix/output_t100.nc") as ds:
    grid = GridInfo(ds)
    lr_sol = LRSol(ds, grid)
    idx_2D = np.array([0, 1])
    P_sum_matrix = lr_sol.marginalDistributions()
    P_slice_matrix = lr_sol.slicedDistributions(slice_vec)

### SSA

In [ ]:
result = np.load("scripts/reference_solutions/bax_ssa_2e6.npy")
ssa_sol = SSASol(result)
P_sum_ssa, _, P_slice_ssa, _ = ssa_sol.calculateObservables(slice_vec, idx_2D)
print(P_slice_ssa[])

In [ ]:
ssa_2e6 = np.load("scripts/reference_solutions/bax_ssa_2e6.npy")
ssa_2e6_sol = SSASol(ssa_2e6)
P_sum_ssa_2e6, _, P_slice_ssa_2e6, _ = ssa_2e6_sol.calculateObservables(slice_vec, idx_2D)

In [ ]:
ssa_1e6 = np.load("scripts/reference_solutions/bax_ssa_1e6.npy")
ssa_1e6_sol = SSASol(ssa_1e6)
P_sum_ssa_1e6, _, P_slice_ssa_1e6, _ = ssa_1e6_sol.calculateObservables(slice_vec, idx_2D)

In [ ]:
ssa_1e5 = np.load("scripts/reference_solutions/bax_ssa_1e5.npy")
ssa_1e5_sol = SSASol(ssa_1e5)
P_sum_ssa_1e5, _, P_slice_ssa_1e5, _ = ssa_1e5_sol.calculateObservables(slice_vec, idx_2D)

## Sliced distributions

In [ ]:
fig, axs = plt.subplots(2, 2, figsize=(7, 5), sharex='col', sharey='row')
axs[0, 0].plot(np.arange(tree.root.child[0].grid.n[0])[11:26], P_slice_tensor0[11:26], '<-', label="PS-TTN integrator ($\mathcal{{P}}_1$)")
axs[0, 0].plot(np.arange(grid.n[0])[11:26], P_slice_matrix[0][11:26], '^', label="DLRA")
axs[0, 0].plot((np.arange(ssa_sol.n[0])+ssa_sol.n_min[0])[8:23], P_slice_ssa[-1][0][8:23], 'v', label="SSA ($10^7$ runs)")
axs[0, 0].set_ylabel("$P_\\mathrm{{S}}(x_0)$")
axs[0, 0].set_yscale("log")

axs[1, 0].plot(np.arange(tree.root.child[0].grid.n[0])[11:26], P_slice_tensor0[11:26], '<-')
axs[1, 0].plot(np.arange(grid.n[0])[11:26], P_slice_matrix[0][11:26], '^')
axs[1, 0].plot((np.arange(ssa_sol.n[0])+ssa_sol.n_min[0])[8:23], P_slice_ssa[-1][0][8:23], 'v')
axs[1, 0].set_xlabel(xlabel="$x_0$")
axs[1, 0].set_ylabel("$P_\\mathrm{{S}}(x_0)$")
axs[1, 0].set_xticks(np.arange(11, 25+2, 2))
axs[1, 0].set_xlim([10, 26])

axs[0, 1].plot(np.arange(tree.root.child[0].grid.n[1])[8:15], P_slice_tensor1[8:15], '<-')
axs[0, 1].plot(np.arange(grid.n[1])[8:15], P_slice_matrix[1][8:15], '^')
axs[0, 1].plot((np.arange(ssa_sol.n[1])+ssa_sol.n_min[1])[8:15], P_slice_ssa[-1][1][8:15], 'v')
axs[0, 1].set_xlabel(xlabel="$x_1$")
axs[0, 1].set_ylabel("$P_\\mathrm{{S}}(x_1)$")
axs[0, 1].set_yscale("log")

axs[1, 1].plot(np.arange(tree.root.child[0].grid.n[1])[8:15], P_slice_tensor1[8:15], '<-')
axs[1, 1].plot(np.arange(grid.n[1])[8:15], P_slice_matrix[1][8:15], '^')
axs[1, 1].plot((np.arange(ssa_sol.n[1])+ssa_sol.n_min[1])[8:15], P_slice_ssa[-1][1][8:15], 'v')
axs[1, 1].set_xlabel(xlabel="$x_1$")
axs[1, 1].set_ylabel("$P_\\mathrm{{S}}(x_1)$")
axs[1, 1].set_xticks(np.arange(8, 15))
axs[1, 1].set_xlim([7, 16])

axs[0, 1].yaxis.tick_right()
axs[0, 1].yaxis.set_ticks_position("both")
axs[0, 1].yaxis.set_label_position("right")
axs[1, 1].yaxis.tick_right()
axs[1, 1].yaxis.set_ticks_position("both")
axs[1, 1].yaxis.set_label_position("right")
fig.legend(*axs[0, 0].get_legend_handles_labels(), ncols=3, loc="upper center")
plt.subplots_adjust(hspace=.0, wspace=.0)

## Marginal distributions

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(9, 4))
ax1.plot(np.arange(tree.root.child[0].grid.n[0]), P_sum_tensor0, '<-', label="PS-TTN integrator ($\mathcal{{P}}_1$)")
ax1.plot(np.arange(grid.n[0]), P_sum_matrix[0], '^', label="DLRA")
ax1.plot((np.arange(ssa_sol.n[0])+ssa_sol.n_min[0]), P_sum_ssa[-1][0], 'v', label="SSA ($10^7$ runs)")
ax1.set_xlabel("$x_0$")
ax1.set_ylabel("$P_\\mathrm{{M}}(x_0)$")

ax2.plot(np.arange(tree.root.child[0].grid.n[1]), P_sum_tensor1, '<-')
ax2.plot(np.arange(grid.n[1]), P_sum_matrix[1], '^')
ax2.plot((np.arange(ssa_sol.n[1])+ssa_sol.n_min[1]), P_sum_ssa[-1][1], 'v')
ax2.set_xlabel("$x_1$")
ax2.yaxis.tick_right()
ax2.yaxis.set_ticks_position("both")
ax2.yaxis.set_label_position("right")
ax2.set_ylabel("$P_\\mathrm{{M}}(x_1)$")
fig.legend(*ax1.get_legend_handles_labels(), ncols=3, loc="upper center");

## Comparison of TTN and SSA results with matrix integrator

In [ ]:
time_series = TimeSeries("output/bax_p1_r5-15-25_i_tau5e-2")
walltime_p1 = time_series.getWallTime()

In [ ]:
time_series = TimeSeries("output/bax_pfull_r5_i_tau5e-2")
walltime_full = time_series.getWallTime()

In [ ]:
walltime_ssa_1e5 = 129
walltime_ssa_1e6 = 358
walltime_ssa_1e7 = 2898
walltime_matrix = 1.3e5

In [ ]:
walltimes = [walltime_matrix, walltime_full, walltime_p1, walltime_ssa_1e7, walltime_ssa_1e6, walltime_ssa_1e5]
labels = ["DLRA", "PS-TTN\n(2 partitions)", "PS-TTN\n($\mathcal{{P}}_1$)",  "SSA\n($10^7$ runs)", "SSA\n($10^6$ runs)", "SSA\n($10^5$ runs)"]

In [ ]:
DLR_marginal_err, SSA_marginal_err = calculateMarginalDistributionError("output/bax_p1_r5-15-25_i_tau5e-2/output_t2900.nc", P_sum_matrix, P_sum_ssa[-1], ssa_sol)

In [ ]:
d = len(ssa_sol.n)
SSA_DLR_marginal_err_1e5 = np.zeros(d)
SSA_DLR_marginal_err_1e6 = np.zeros(d)
SSA_DLR_marginal_err_2e6 = np.zeros(d)
for i in range(d):
    SSA_DLR_marginal_err_1e5[i] = np.linalg.norm(P_sum_matrix[i][ssa_1e5_sol.n_min[i] : ssa_1e5_sol.n_min[i]+ssa_1e5_sol.n[i]] - P_sum_ssa_1e5[-1][i][:grid.n[i]], np.inf)
    SSA_DLR_marginal_err_1e6[i] = np.linalg.norm(P_sum_matrix[i][ssa_1e6_sol.n_min[i] : ssa_1e6_sol.n_min[i]+ssa_1e6_sol.n[i]] - P_sum_ssa_1e6[-1][i][:grid.n[i]], np.inf)
    SSA_DLR_marginal_err_2e6[i] = np.linalg.norm(P_sum_matrix[i][ssa_2e6_sol.n_min[i] : ssa_2e6_sol.n_min[i]+ssa_2e6_sol.n[i]] - P_sum_ssa_2e6[-1][i][:grid.n[i]], np.inf)

### Plots

In [ ]:
time_series = TimeSeries("output/bax_p1_r5-15-25_i_tau5e-2")
mass_err = np.abs(time_series.getMassErr())
fig, ax = plt.subplots()
ax.plot(time_series.time, mass_err, '-')
ax.set_xlabel("time $t$")
ax.set_ylabel("$\Delta m$")
ax.set_yscale("log")
ax.set_ylim([8e-9, 1e-2]);

In [ ]:
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(6, 7))
ax1.plot(np.arange(grid.d), DLR_marginal_err, '<-', label="PS-TTN ($\mathcal{{P}}_1$)")
ax1.plot(np.arange(grid.d), SSA_DLR_marginal_err_1e5, '^-', label="SSA ($10^5$ runs)")
ax1.plot(np.arange(grid.d), SSA_DLR_marginal_err_1e6, 'v-', label="SSA ($10^6$ runs)")
ax1.plot(np.arange(grid.d), SSA_DLR_marginal_err_2e6, '>-', label="SSA ($10^7$ runs)")
ax1.set_yscale("log")
ax1.set_xlabel("species $S_i$")
ax1.set_ylabel("$\max_{{x_i}}|P_M(x_i)-P_M(x_i)^\mathrm{{DLRA}}|$")
ax2.bar(labels, walltimes)
ax2.set_yscale("log")
ax2.set_ylabel("wall time [$\mathrm{{s}}$]")
plt.subplots_adjust(hspace=.3)
fig.legend(loc="upper center", ncol=4);

In [ ]:
d = len(ssa_sol.n)
SSA_marginal_err_1e5 = np.zeros(d)
SSA_marginal_err_1e6 = np.zeros(d)
for i in range(d):
    # SSA_marginal_err_1e5[i] = np.linalg.norm(P_sum_ssa[-1][i][ssa_1e5_sol.n_min[i] : ssa_1e5_sol.n_min[i]+ssa_1e5_sol.n[i]] - P_sum_ssa_1e5[-1][i][ssa_sol.n_min[i] : ssa_sol.n_min[i]+ssa_sol.n[i]], np.inf)
    # SSA_marginal_err_1e6[i] = np.linalg.norm(P_sum_ssa[-1][i][ssa_1e6_sol.n_min[i] : ssa_1e6_sol.n_min[i]+ssa_1e6_sol.n[i]] - P_sum_ssa_1e6[-1][i][ssa_sol.n_min[i] : ssa_sol.n_min[i]+ssa_sol.n[i]], np.inf)

## Error depending on time step size

In [ ]:
dm_max = []
tau = []

time_series = TimeSeries("output/bax_p1_r5-15-15_i_tau5e-1")
dm_max.append(np.abs(time_series.getMaxMassErr()))
tau.append(time_series.getTau())

time_series = TimeSeries("output/bax_p1_r5-15-15_i_tau2e-1")
dm_max.append(np.abs(time_series.getMaxMassErr()))
tau.append(time_series.getTau())

time_series = TimeSeries("output/bax_p1_r5-15-15_i_tau1e-1")
dm_max.append(np.abs(time_series.getMaxMassErr()))
tau.append(time_series.getTau())

time_series = TimeSeries("output/bax_p1_r5-15-15_i_tau5e-2")
dm_max.append(np.abs(time_series.getMaxMassErr()))
tau.append(time_series.getTau())

time_series = TimeSeries("output/bax_p1_r5-15-15_i_tau2e-2")
dm_max.append(np.abs(time_series.getMaxMassErr()))
tau.append(time_series.getTau())

time_series = TimeSeries("output/bax_p1_r5-15-15_i_tau1e-2")
dm_max.append(np.abs(time_series.getMaxMassErr()))
tau.append(time_series.getTau())

### Plot

In [ ]:
fig, ax = plt.subplots()
t = np.linspace(0.01, 0.5)

ax.loglog(tau, dm_max, '.-', label="$\max(\Delta m)$")
ax.loglog(t, t**3*10, 'k:', label="$\\tau^3$")
ax.set_xlabel("time step size $\\tau$")
fig.legend(loc="upper center", ncol=2);

## Comparison with deterministic solution

In [ ]:
time_series = TimeSeries("output/bax_p1_r5-15-25_i_tau5e-2")
concentrations = time_series.calculateMoments()
t = time_series.time

In [ ]:
from pysb.integrate import odesolve
from scripts.models.bax_pysb import model
concentrations_ode = odesolve(model, t)

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(7, 4))
deviation = np.sqrt(concentrations[1]-concentrations[0]**2)
observables = ["oBax1", "oBax2"]
observables_alt = ["$S_0$", "$S_1$", "$S_2$"]
colors = plt.rcParams['axes.prop_cycle'].by_key()['color']
for i, (o, o_alt) in enumerate(zip(observables, observables_alt)):
    ax1.plot(t, concentrations[0][:, i], '-', label=o_alt, color=colors[i], alpha=0.7)
    ax1.fill_between(t, concentrations[0][:, i]-deviation[:, i], concentrations[0][:, i]+deviation[:, i], color=colors[i], alpha=.2)
    ax1.plot(t, concentrations_ode[o], '--', color=colors[i], alpha=1.0)
ax1.set_ylabel("$\langle x_i(t) \\rangle$")
ax1.set_ylim([0.0, 45.0])

observables = ["oBax3", "oBax4", "oBax5"]
observables_alt = ["$S_2$", "$S_3$", "$S_4$"]
for idx_o, (o, o_alt) in enumerate(zip(observables, observables_alt)):
    i = idx_o + 2
    ax2.plot(t, concentrations[0][:, i], '-', label=o_alt, color=colors[i], alpha=0.7)
    ax2.fill_between(t, concentrations[0][:, i]-deviation[:, i], concentrations[0][:, i]+deviation[:, i], color=colors[i], alpha=.2)
    ax2.plot(t, concentrations_ode[o], '--', color=colors[i], alpha=1.0)
ax2.set_ylim([0.0, 4])
ax2.yaxis.tick_right()
ax2.yaxis.set_ticks_position("both")
ax2.yaxis.set_label_position("right")
plt.setp((ax1, ax2), xlabel="$t$", xlim=[0.0, 145.0], xticks=[0.0, 50.0, 100.0, 145.0]);

lines_labels = [ax.get_legend_handles_labels() for ax in fig.axes]
lines, labels = [sum(ll, []) for ll in zip(*lines_labels)]
fig.legend(lines, labels, ncols=5, loc="upper center");

In [ ]:
fig, ax = plt.subplots(figsize=(6, 4), layout='constrained')
for i, o in enumerate(model.observables):
    ax.plot(t, np.abs((concentrations[0][:, i]-concentrations_ode[o.name])), '-', label="$S_{{{}}}$".format(i))
ax.set_xlabel("$t$")
ax.set_ylabel("$|c_i^\mathrm{{CME}}(t) - c_i^\mathrm{{ODE}}(t)|$");
for line in ax.get_lines()[0:4]:
    line.set_linestyle('-')
for line in ax.get_lines()[4:8]:
    line.set_linestyle('--')
for line in ax.get_lines()[8:]:
    line.set_linestyle('-.')
fig.legend(*ax.get_legend_handles_labels(), loc="outside center right");

## Error depending on rank and partition

### Load data

#### Partition 0

In [ ]:
DLR_marginal_err00, SSA_marginal_err00 = calculateMarginalDistributionError("output/bax_p0_r5-5-5_i_tau5e-2/output_t2900.nc", P_sum_matrix, P_sum_ssa[-1], ssa_sol)

DLR_marginal_err10, SSA_marginal_err10 = calculateMarginalDistributionError("output/bax_p0_r5-10-10_i_tau5e-2/output_t2900.nc", P_sum_matrix, P_sum_ssa[-1], ssa_sol)

DLR_marginal_err20, SSA_marginal_err20 = calculateMarginalDistributionError("output/bax_p0_r5-15-15_i_tau5e-2/output_t2900.nc", P_sum_matrix, P_sum_ssa[-1], ssa_sol)

DLR_marginal_err30, SSA_marginal_err30 = calculateMarginalDistributionError("output/bax_p0_r5-20-20_i_tau5e-2/output_t2900.nc", P_sum_matrix, P_sum_ssa[-1], ssa_sol)

#### Partition 1

In [ ]:
DLR_marginal_err01, SSA_marginal_err01 = calculateMarginalDistributionError("output/bax_p1_r5-5-5_i_tau5e-2/output_t2900.nc", P_sum_matrix, P_sum_ssa[-1], ssa_sol)

DLR_marginal_err11, SSA_marginal_err11 = calculateMarginalDistributionError("output/bax_p1_r5-10-10_i_tau5e-2/output_t2900.nc", P_sum_matrix, P_sum_ssa[-1], ssa_sol)

DLR_marginal_err21, SSA_marginal_err21 = calculateMarginalDistributionError("output/bax_p1_r5-15-15_i_tau5e-2/output_t2900.nc", P_sum_matrix, P_sum_ssa[-1], ssa_sol)

DLR_marginal_err31, SSA_marginal_err31 = calculateMarginalDistributionError("output/bax_p1_r5-20-20_i_tau5e-2/output_t2900.nc", P_sum_matrix, P_sum_ssa[-1], ssa_sol)

#### Partition 2

In [ ]:
DLR_marginal_err02, SSA_marginal_err02 = calculateMarginalDistributionError("output/bax_p2_r5-5-5_i_tau5e-2/output_t2900.nc", P_sum_matrix, P_sum_ssa[-1], ssa_sol)

DLR_marginal_err12, SSA_marginal_err12 = calculateMarginalDistributionError("output/bax_p2_r5-10-10_i_tau5e-2/output_t2900.nc", P_sum_matrix, P_sum_ssa[-1], ssa_sol)

DLR_marginal_err22, SSA_marginal_err22 = calculateMarginalDistributionError("output/bax_p2_r5-15-15_i_tau5e-2/output_t2900.nc", P_sum_matrix, P_sum_ssa[-1], ssa_sol)

DLR_marginal_err32, SSA_marginal_err32 = calculateMarginalDistributionError("output/bax_p2_r5-20-20_i_tau5e-2/output_t2900.nc", P_sum_matrix, P_sum_ssa[-1], ssa_sol)

#### Two partitions

In [ ]:
DLR_marginal_err_full, SSA_marginal_err_full = calculateMarginalDistributionError("output/bax_pfull_r5_i_tau5e-2/output_t2900.nc", P_sum_matrix, P_sum_ssa[-1], ssa_sol)

#### SSA with $10^6$ samples

In [ ]:
ssa_1e5 = np.load("scripts/reference_solutions/bax_ssa_1e5.npy")
ssa_1e5_sol = SSASol(ssa_1e5)
P_sum_ssa_1e5, _, _, _ = ssa_1e5_sol.calculateObservables(slice_vec, idx_2D)

ssa_1e6 = np.load("scripts/reference_solutions/bax_ssa_1e6.npy")
ssa_1e6_sol = SSASol(ssa_1e6)
P_sum_ssa_1e6, _, _, _ = ssa_1e6_sol.calculateObservables(slice_vec, idx_2D)

DLR_marginal_err_ssa_1e5 = np.zeros(tree.grid.d())
SSA_marginal_err_ssa_1e5 = np.zeros(tree.grid.d())
DLR_marginal_err_ssa_1e6 = np.zeros(tree.grid.d())
SSA_marginal_err_ssa_1e6 = np.zeros(tree.grid.d())

for i in range(tree.grid.d()):
    DLR_marginal_err_ssa_1e5[i] = np.linalg.norm(P_sum_ssa_1e5[-1][i][:tree.grid.n[i]] - P_sum_matrix[i][ssa_1e5_sol.n_min[i] : ssa_1e5_sol.n_min[i]+ssa_1e5_sol.n[i]], np.inf)
    DLR_marginal_err_ssa_1e6[i] = np.linalg.norm(P_sum_ssa_1e6[-1][i][:tree.grid.n[i]] - P_sum_matrix[i][ssa_1e6_sol.n_min[i] : ssa_1e6_sol.n_min[i]+ssa_1e6_sol.n[i]], np.inf)

### Plot

In [ ]:
# from mpl_toolkits.axes_grid1.inset_locator import inset_axes
colors = plt.rcParams['axes.prop_cycle'].by_key()['color']
cols = ["$\mathcal{{P}}_0$", "$\mathcal{{P}}_1$", "$\mathcal{{P}}_2$"]
rows = ["$r = (5, 5, 5)$", "$r = (5, 10, 10)$", "$r = (5, 15, 15)$", "$r = (5, 20, 20)$"]
labels = ["$\max_{{x_i}}|P_M(x_i)^\mathrm{{TTN}}-P_M(x_i)^\mathrm{{SSA}}|$", 
          "$\max_{{x_i}}|P_M(x_i)^\mathrm{{TTN}}-P_M(x_i)^\mathrm{{DLRA}}|$"]

fig, axs = plt.subplots(4, 3, figsize=(10, 8), sharex='col', sharey='row')

line0, = axs[0, 0].plot(np.arange(tree.grid.d()), SSA_marginal_err00, '--', color=colors[0])
line1, = axs[0, 0].plot(np.arange(tree.grid.d()), DLR_marginal_err00, '.-', color=colors[0])
axs[1, 0].plot(np.arange(tree.grid.d()), SSA_marginal_err10, '--', color=colors[0])
axs[1, 0].plot(np.arange(tree.grid.d()), DLR_marginal_err10, '.-', color=colors[0])
axs[2, 0].plot(np.arange(tree.grid.d()), SSA_marginal_err20, '--', color=colors[0])
axs[2, 0].plot(np.arange(tree.grid.d()), DLR_marginal_err20, '.-', color=colors[0])
axs[3, 0].plot(np.arange(tree.grid.d()), SSA_marginal_err30, '--', color=colors[0])
axs[3, 0].plot(np.arange(tree.grid.d()), DLR_marginal_err30, '.-', color=colors[0])

axs[0, 1].plot(np.arange(tree.grid.d()), SSA_marginal_err01, '--', color=colors[1])
axs[0, 1].plot(np.arange(tree.grid.d()), DLR_marginal_err01, '.-', color=colors[1])
axs[1, 1].plot(np.arange(tree.grid.d()), SSA_marginal_err11, '--', color=colors[1])
axs[1, 1].plot(np.arange(tree.grid.d()), DLR_marginal_err11, '.-', color=colors[1])
axs[2, 1].plot(np.arange(tree.grid.d()), SSA_marginal_err21, '--', color=colors[1])
axs[2, 1].plot(np.arange(tree.grid.d()), DLR_marginal_err21, '.-', color=colors[1])
axs[3, 1].plot(np.arange(tree.grid.d()), SSA_marginal_err31, '--', color=colors[1])
axs[3, 1].plot(np.arange(tree.grid.d()), DLR_marginal_err31, '.-', color=colors[1])

axs[0, 2].plot(np.arange(tree.grid.d()), SSA_marginal_err02, '--', color=colors[2])
axs[0, 2].plot(np.arange(tree.grid.d()), DLR_marginal_err02, '.-', color=colors[2])
axs[1, 2].plot(np.arange(tree.grid.d()), SSA_marginal_err12, '--', color=colors[2])
axs[1, 2].plot(np.arange(tree.grid.d()), DLR_marginal_err12, '.-', color=colors[2])
axs[2, 2].plot(np.arange(tree.grid.d()), SSA_marginal_err22, '--', color=colors[2])
axs[2, 2].plot(np.arange(tree.grid.d()), DLR_marginal_err22, '.-', color=colors[2])
axs[3, 2].plot(np.arange(tree.grid.d()), SSA_marginal_err32, '--', color=colors[2])
axs[3, 2].plot(np.arange(tree.grid.d()), DLR_marginal_err32, '.-', color=colors[2])

# axins = inset_axes(axs[-1, 1], width='65%', height='65%', loc='upper center')
# axins.plot(np.arange(tree.grid.d()), SSA_marginal_err_full, '--')
# axins.plot(np.arange(tree.grid.d()), DLR_marginal_err_full, '.-')
# axins.set_ylim([-0.02*0.6, 0.42*0.6])
# axins.annotate("TTN with 2 partitions\n($r=5$)", xy=(0.5, 1), xytext=(0, -30),
#                 xycoords='axes fraction', textcoords='offset points',
#                 size=11, ha='center', va='baseline')

plt.setp(axs, ylim=[-0.02, 0.42])
# plt.setp(axs[:, 0], ylim=[-0.02, 0.22])
# plt.setp(axs[:, 1], ylim=[-0.02, 0.47])
# plt.setp(axs[:, 2], ylim=[-0.02, 0.34])

pad = 5 # in points

for ax, col in zip(axs[0], cols):
    ax.annotate(col, xy=(0.5, 1), xytext=(0, pad+5),
                xycoords='axes fraction', textcoords='offset points',
                size=14, ha='center', va='baseline')

for ax, row in zip(axs[:,0], rows):
    ax.annotate(row, xy=(0, 0.5), xytext=(-ax.yaxis.labelpad-pad, 0),
                xycoords=ax.yaxis.label, textcoords='offset points',
                size=14, ha='right', va='center')

plt.setp(axs[-1, :], xlabel="species $S_i$", xticks=[0, 2, 4, 6, 8, 10])
# fig.suptitle(labels[1])
fig.legend([line0, line1], labels, ncols=2, loc="upper center");
plt.subplots_adjust(hspace=.0, wspace=.0)

## Mass error

### Load data

#### Partition 0

In [ ]:
time_series = TimeSeries("output/bax_p0_r5-5-5_i_tau5e-2")
mass_err00 = np.abs(time_series.getMassErr())
time = time_series.time

time_series = TimeSeries("output/bax_p0_r5-10-10_i_tau5e-2")
mass_err10 = np.abs(time_series.getMassErr())

time_series = TimeSeries("output/bax_p0_r5-15-15_i_tau5e-2")
mass_err20 = np.abs(time_series.getMassErr())

time_series = TimeSeries("output/bax_p0_r5-20-20_i_tau5e-2")
mass_err30 = np.abs(time_series.getMassErr())

#### Partition 1

In [ ]:
time_series = TimeSeries("output/bax_p1_r5-5-5_i_tau5e-2")
mass_err01 = np.abs(time_series.getMassErr())

time_series = TimeSeries("output/bax_p1_r5-10-10_i_tau5e-2")
mass_err11 = np.abs(time_series.getMassErr())

time_series = TimeSeries("output/bax_p1_r5-15-25_i_tau5e-2")
mass_err21 = np.abs(time_series.getMassErr())

time_series = TimeSeries("output/bax_p1_r5-20-20_i_tau5e-2")
mass_err31 = np.abs(time_series.getMassErr())

#### Partition 2

In [ ]:
time_series = TimeSeries("output/bax_p2_r5-5-5_i_tau5e-2")
mass_err02 = np.abs(time_series.getMassErr())

time_series = TimeSeries("output/bax_p2_r5-10-10_i_tau5e-2")
mass_err12 = np.abs(time_series.getMassErr())

time_series = TimeSeries("output/bax_p2_r5-15-15_i_tau5e-2")
mass_err22 = np.abs(time_series.getMassErr())

time_series = TimeSeries("output/bax_p2_r5-20-20_i_tau5e-2")
mass_err32 = np.abs(time_series.getMassErr())

### Plot

In [ ]:
fig, axs = plt.subplots(2, 2, figsize=(7, 5))

axs[0, 0].plot(time, mass_err00, label="$\mathcal{{P}}_0$")
axs[0, 0].plot(time, mass_err01, label="$\mathcal{{P}}_1$")
axs[0, 0].plot(time, mass_err02, label="$\mathcal{{P}}_2$")
axs[0, 0].set_title("$r = (5, 5, 5)$")

axs[0, 1].plot(time, mass_err10)
axs[0, 1].plot(time, mass_err11)
axs[0, 1].plot(time, mass_err12)
axs[0, 1].set_title("$r = (5, 10, 10)$")
axs[0, 1].yaxis.tick_right()
axs[0, 1].yaxis.set_ticks_position("both")
axs[0, 1].yaxis.set_label_position("right")

axs[1, 0].plot(time, mass_err20)
axs[1, 0].plot(time, mass_err21)
axs[1, 0].plot(time, mass_err22)
axs[1, 0].set_title("$r = (5, 15, 15)$")

axs[1, 1].plot(time, mass_err30)
axs[1, 1].plot(time, mass_err31)
axs[1, 1].plot(time, mass_err32)
axs[1, 1].set_title("$r = (5, 20, 20)$")
axs[1, 1].yaxis.tick_right()
axs[1, 1].yaxis.set_ticks_position("both")
axs[1, 1].yaxis.set_label_position("right")

plt.setp(axs, xlabel="time $t$", ylabel="$\Delta m$", xlim=[0.0, 145.0], xticks=[0.0, 50.0, 100.0, 145.0], ylim=[8e-9, 1e-2], yscale="log")
plt.subplots_adjust(hspace=0.5)
fig.legend(*axs[0, 0].get_legend_handles_labels(), ncols=3, loc="upper center");

## Wall time

### Load data

#### Partition 0

In [ ]:
time_series = TimeSeries("output/bax_p0_r5-5-5_i_tau5e-2")
walltime00 = time_series.getWallTime()

time_series = TimeSeries("output/bax_p0_r5-10-10_i_tau5e-2")
walltime10 = time_series.getWallTime()

time_series = TimeSeries("output/bax_p0_r5-15-15_i_tau5e-2")
walltime20 = time_series.getWallTime()

time_series = TimeSeries("output/bax_p0_r5-20-20_i_tau5e-2")
walltime30 = time_series.getWallTime()

#### Partition 1

In [ ]:
time_series = TimeSeries("output/bax_p1_r5-5-5_i_tau5e-2")
walltime01 = time_series.getWallTime()

time_series = TimeSeries("output/bax_p1_r5-10-10_i_tau5e-2")
walltime11 = time_series.getWallTime()

time_series = TimeSeries("output/bax_p1_r5-15-15_i_tau5e-2")
walltime21 = time_series.getWallTime()

time_series = TimeSeries("output/bax_p1_r5-20-20_i_tau5e-2")
walltime31 = time_series.getWallTime()

#### Partition 2

In [ ]:
time_series = TimeSeries("output/bax_p2_r5-5-5_i_tau5e-2")
walltime02 = time_series.getWallTime()

time_series = TimeSeries("output/bax_p2_r5-10-10_i_tau5e-2")
walltime12 = time_series.getWallTime()

time_series = TimeSeries("output/bax_p2_r5-15-15_i_tau5e-2")
walltime22 = time_series.getWallTime()

time_series = TimeSeries("output/bax_p2_r5-20-20_i_tau5e-2")
walltime32 = time_series.getWallTime()

### Plot

In [ ]:
fig, axs = plt.subplots(2, 2, figsize=(7, 5))
labels = ["$\mathcal{{P}}_{}$".format(i) for i in range(3)]
colors = plt.rcParams['axes.prop_cycle'].by_key()['color']
walltime0 = [walltime00, walltime01, walltime02]
axs[0, 0].bar(labels, walltime0, color=colors)
axs[0, 0].set_title("$r = (5, 5, 5)$")

walltime1 = [walltime10, walltime11, walltime12]
axs[0, 1].bar(labels, walltime1, color=colors)
axs[0, 1].set_title("$r = (5, 10, 10)$")
axs[0, 1].yaxis.tick_right()
axs[0, 1].yaxis.set_ticks_position("both")
axs[0, 1].yaxis.set_label_position("right")

walltime2 = [walltime20, walltime21, walltime22]
axs[1, 0].bar(labels, walltime2, color=colors)
axs[1, 0].set_title("$r = (5, 15, 15)$")

walltime3 = [walltime30, walltime31, walltime32]
axs[1, 1].bar(labels, walltime3, color=colors)
axs[1, 1].set_title("$r = (5, 20, 20)$")
axs[1, 1].yaxis.tick_right()
axs[1, 1].yaxis.set_ticks_position("both")
axs[1, 1].yaxis.set_label_position("right")

plt.setp(axs[:, 0], ylabel="wall time [$\mathrm{s}$]")
# plt.setp(axs, ylim=[1e0, 1e6], yscale="log")
plt.subplots_adjust(hspace=0.3)
